# DAGs - a generalization of the alignment problem
##Specification
- Name: problem15 in Rosalind
- Name your notebook as: DAG.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it handy to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind5.txt.out ( for example).
-Rosalind Problem Name: Find the longest path in a DAG

As always, include a Inspection Intro Markdown that describes your specific algorithm at the beginning of the notebook, and another Inspection Results markdown at the end of the notebook that documents: your inspection team, the findings of the team, and your resolution of those findings.

Please submit your notebook, an example of one of the Rosalind files that you ran and passed, and the output that your program generated as a text file.

## Description
As a prelude to an upcoming assignment in profile alignments, please consider the related problem of generalized weighted-DAGs (directed acyclic graph). Rosalind has a very nice example that includes weighted edges connecting nodes. As applied to a simple pairwise alignment, nodes represent a specific assignment of a character pairing between a character in one of the sequences to either an initial insert/delete (indel), an extension of that indel, or a match/mismatch. 

Unlike classical pairwise alignment, the specific order to consider the resolution of scores in the graph is established using the topology of the graph ( topological ordering). This ordering will avoid recursion in developing the final score. This is another way of using Dynamic programming in a more generalized way. ( pp247-52 in Compeau and the detour on pp287-88).

## Hints
1) scoring of nodes that have no inputs (or possibly incomplete inputs) must be done carefully. Choose these wisely. In a pairwise alignment, these nodes would be considered boundary nodes.

2) The problem is quite clear on the starting/ending node to consider. Make sure to evaluate these properly.

3) each node can have many inputs or none. In the generalized DAG, there is no limit on the number of inputs. Consider a data structure here that allows you to represent the incoming edge along with a score that is computed by adding the weight of that incoming edge to the score of its preceding node.

4) As you write these classes, consider that these may be usable in the future.

5) sets and dictionaries require keys that are immutable. Those keys can be integers, or floats, or tuples. Think carefully about using tuples as containers-that-can-be-keys.

6) python has a container called a namedtuple. These are available in the container module. They are quite handy.

7) python has two functions called any() and all(); any() evaluates to true if any of its iterable arguments are True; all() evaluates to True if all of its iterable arguments are true. None, for example, evaluates to False. 

8) consider implementing a helper method called _argMax_. If you happen to have a dictionary of values, it returns the key whose paired-value is maximal among all values in the dictionary.

9) Carefully consider the implementations that are provided in the text. Specifically, the topology ordering pseudocode removes edges from the constructed DAG. This is a bit annoying, considering that we need that DAG for scoring and backtracking. What is the purpose that removing an edge is attempting to perform? Look over the Wikipedia article on "Dataflow" for an idea as to how you might avoid destroying your graph.

10) My implementation has specific objects that are nodes, edges, and the DAG itself. It is implemented in about 120 lines including whitespace.

11) Rosalind will provide test cases where the proscribed beginning and ending node have inputs/outputs that may induce cycles. In these problems, remember that we are considering the DAG that is described between (inclusively) the start and end node so the inputs to start don't matter; the outputs from end don't matter either.

 

Here is a template to use.

## Inspection Intro

My approach begins with the creation of essential dictionaries, followed by an exploration of all potential paths between the starting and ending nodes. Subsequently, I introduce a separate function responsible for scoring these paths and storing them in a list. Among the gathered scores, I identify the maximum value and its associated path. This information allows me to present the highest score achieved and reveal the longest path found within the graph.

Paul helped figure out implement the longest path function as I was strugling to understand how to get the maximum score and its subsequent path

In [74]:
class DAG:
    """
     A class for working with Directed Acyclic Graphs (DAGs) and finding the longest path.
     
     Attributes:
        source (str): The starting node in the DAG.
        sink (str): The ending node in the DAG.
        lines (list): A list of graph elements in the format "from->to:weight".
        node_dict (dict): A dictionary to store nodes and their possible outgoing edges.
        edge_dict (dict): A dictionary to store edges and their corresponding weights.
        acyclic_path (list): A list to store all possible paths.
        weight (list): A list to store the weights of all possible paths.


    """
    
    def __init__(self, start, end, contents):
        """
        Determine the lengthiest route within a Directed Acyclic Graph and provide the associated score.
        """
        self.source = start
        self.sink = end
        self.lines = contents
        self.node_dict = {} 
        self.edge_dict = {} 
        self.acyclic_path = []
        self.weight = []

    def parseInput(self):
        """
        Generate dictionaries for nodes and their potential outgoing edges,
        and for edges and their corresponding weights.
        """
        
        for line in self.lines:
            arrow = line.split(":")[0]
            splitting = line.split("->")
            out_edge, weight = splitting[1].split(":")
            tuple = (arrow)
            value = (splitting[0])
            if (out_edge == self.source):
                continue
            elif (value == self.sink):
                continue
            else:
                if value in self.node_dict.keys():
                    self.node_dict[value].append((out_edge))
                else:
                    self.node_dict[value] = [out_edge]
                self.edge_dict[tuple] = (int(weight))

        return self.node_dict, self.edge_dict

    def topological_sort(self, this, end, path = []):
        """
        The topological_sort method performs a topological sort of the nodes in the graph. 
        It uses Depth-First Search (DFS) to traverse the graph, starting from unvisited nodes, removing and appending nodes to a stack 
        as they are processed.
        
        """
        for flow in self.node_dict.get(this):
            direct = path + [flow]
            this = direct[-1]
            if direct[-1] == end:
                self.acyclic_path.append(direct)
            else:
                if this not in self.node_dict.keys():
                    self.node_dict[direct[-2]].remove(direct[-1])
                    direct.pop()
                self.topological_sort(direct[-1], end, direct) 
        return self.acyclic_path

    def longestPath(self):
        """
        The find_longest_path function finds the longest path in a directed acyclic graph by calculating the weights of each path and displays the
        maximum score and most extended route.
        """
        weights = []
        for element in self.acyclic_path:
            count = 0 
            for ele in range(1,len(element)):
                vertex = str(element[ele-1])
                svalue = str(element[ele])
                vertex += "->"
                vertex += svalue
                if vertex in self.edge_dict:
                    x = self.edge_dict[vertex]
                    count += x
            weights.append(count)
        max_element = max(weights)
        other = weights.index(max_element)
        longest_path = "->".join(self.acyclic_path[other])
        print(max_element)
        print(longest_path)

def main(inFile=None):
    """
    Reads the input file and calls the necessary functions to display the longest path and highest score.
    """
    with open(inFile) as fh:
        text = []
        for line in fh:
            text.append(line.strip())
        source = (text[0])
        sink = (text[1])
        input_edges = text[2:]
    dag_obj = DAG(source, sink, input_edges)
    dag_obj.parseInput()
    dag_obj.topological_sort(source, sink, [source])
    dag_obj.longestPath()

if __name__ == "__main__":
    main(inFile='rosalind_ba5d.txt')

229
0->6->10->11->16->20->22->23->25
